In [ ]:
import pandas as pd
import numpy as np
import wrf
import glob
import os
import netCDF4 as nc
import xarray as xr
import matplotlib.pyplot as plt
from geographiclib.geodesic import Geodesic
Geo = Geodesic.WGS84

In [ ]:
df_2004 = pd.read_csv('naad_checked_candidates_2000_2004.csv')
df_2004 = df_2004[df_2004['selection'] != 2]
df_2005 = pd.read_csv('tracks_new.csv')

In [ ]:
df_2005.rename(columns={'datetime':'time','lon0':'cand_lon','lat0':'cand_lat'}, inplace=True)

In [ ]:
df_2004_1 = df_2004[['time','cand_lon','cand_lat', 'diam']]
df_2005_1 = df_2005[['time','cand_lon','cand_lat', 'diam']]

In [ ]:
df_concat = pd.concat([df_2004_1,df_2005_1])

In [ ]:
df_concat.sort_values(by='time', inplace=True)

In [ ]:
filedir = '../Data/originals/'
filename = []
filename += glob.glob(filedir+os.sep+'*_cut.nc')

ncdf = nc.Dataset(filename[0],engine='netcdf4')
lons = wrf.getvar(ncdf, 'XLONG')
lats = wrf.getvar(ncdf, 'XLAT')

diams = df_concat['diam']

In [ ]:
atm_times = np.load('times_all.npy')
atm_times = np.reshape(atm_times, 3551*8)

atm_times_pd = []
for i in range(atm_times.shape[0]):
    atm_times_pd.append(pd.Timestamp(atm_times[i]))

In [ ]:
mask = xr.open_dataset('../Data/NAAD14km_lsm.nc')

mask_lat = (mask.south_north>=467)&(mask.south_north<=549)
mask_lon = (mask.west_east>=317)&(mask.west_east<=478)

masked_mask = mask.where(mask_lon & mask_lat, drop=True)

lats_ = masked_mask['XLAT'].where(masked_mask.lsm==0)[0]
lons_ = masked_mask['XLONG'].where(masked_mask.lsm==0)[1]
lats1 = lats_[~np.isnan(lats_)]
lons1 = lons_[~np.isnan(lons_)]

In [ ]:
false_sample = pd.DataFrame(index = [i for i in range(1000)], columns = df_concat.columns)

In [ ]:
def create_false_sample(false_sample):
    false_sample.loc[i,'time'] = np.random.choice(atm_times_pd)
    false_sample.loc[i,'cand_lon'] = np.random.choice(lons1[30:len(lats1)-30])
    false_sample.loc[i,'cand_lat'] = np.random.choice(lats1[30:len(lats1)-30])
    
    lon_min = -27.85531
    lon_max = 4.829739
    lat_min = 61.794632
    lat_max = 68.39437
    
    lat_min_diff = false_sample.loc[i,'cand_lat'] - lat_min
    lat_max_diff = lat_max - false_sample.loc[i,'cand_lat']
    lon_max_diff = lon_max - false_sample.loc[i,'cand_lon']
    lon_min_diff = false_sample.loc[i,'cand_lon'] - lon_min
    
    border = np.min([lat_min_diff, lat_max_diff, lon_max_diff, lon_min_diff]) * 2 * 112
    
    if border>1000:
        border =1000
        false_sample.loc[i,'diam'] = np.random.choice(np.arange(600,border,1))
    elif border>700:
        false_sample.loc[i,'diam'] = np.random.choice(np.arange(400,border,1))
    elif border>400:
        false_sample.loc[i,'diam'] = np.random.choice(np.arange(300,border,1))
    
    false_sample.loc[i,'diam'] = np.random.choice(np.arange(200,border,0.1))

In [ ]:
for i in false_sample.index:
    create_false_sample(false_sample)

In [ ]:
false_sample.to_csv('test_false_sample.csv')